In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn import linear_model,metrics,preprocessing
from sklearn import ensemble,tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.neighbors import RadiusNeighborsClassifier

Hello reader,

This exercise is good for beginners who are learning about Overfitting and its consequences to the overall predictions. Efforts are being made to analyze importance of Cross-Validation on a dataset and its effect upon removing the Overfitting in a data.Also the importance of Data visualization has been discussed here.It is often a misconception, where we avoid to look at the distribution of a data, leading to failure of models at production.
To forefit such issues, a naive approach of splitting the model for training-testing followed by an in-depth discussion of the distribution of the data has been done here.

Please don't forget to upvote this Notebook :D

In [ ]:
df1= pd.read_csv('/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')

Clearly this data represents 11 parameters on which the quality of a Red Wine depends.
We have a task here to predict the quality of a certain type of wine from 3 to 8.
The better the quality, the more likeability for its consumption :)

In [ ]:
df1.head()

In [ ]:
#we map the quality values from 0 to 5
qual_map={
    3:0,
    4:1,
    5:2,
    6:3,
    7:4,
    8:5
}
df1.loc[:,"quality"]=df1.quality.map(qual_map)

In [ ]:
#For our convenience we have mapped the quality indexes from 3-8 to 0-5, and shall treat this as a multi-class 
#classification problem.
df1


In [ ]:
#Naively splitting data without visualization 
#using dataframe.sample for random sampling of the data.We will reset the indices,as they change after 
#shuffling the data.

df1= df1.sample(frac=1).reset_index(drop=True)

#top 1000 out of 1600 rows are our training data.
df1_train=df1.head(1000)

#bottom 600 out of 1600 rows are our testing data.
df1_test=df1.tail(600)




In [ ]:
df1_train

In [ ]:
df1_test

In [ ]:
#Now as this problem can be treated as a classification task, we train this using Decision Tree model
import sklearn
from sklearn import tree
from sklearn import metrics

classifier= tree.DecisionTreeClassifier(max_depth=3)

cols=['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']
classifier.fit(df1_train[cols],df1_train.quality)

Predicting the Decision tree model created above on the data.

In [ ]:
train_predictions= classifier.predict(df1_train[cols])

test_predictions= classifier.predict(df1_test[cols])

In [ ]:
train_accuracy= metrics.accuracy_score(df1_train.quality,train_predictions)

test_accuracy= metrics.accuracy_score(df1_test.quality,test_predictions)


Note below , that the train_accuracy and test_accuracy for a negligble hyperparameter tuning, i.e. using 
max_depth=3 for the decision tree, gives these results. 

We now try to see the variations in training and testing accuracies when the max_depth for the Decision tree is 
tuned.

In [ ]:
train_accuracy

In [ ]:
test_accuracy

In [ ]:
#Importing libraries for visualization.
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Global size of texts on the plots.
matplotlib.rc('xtick', labelsize=15)
matplotlib.rc('ytick', labelsize=15)


In [ ]:
#To ensure that plot is displayed inside the notebook itself
%matplotlib inline

In [ ]:
#initializing the list to store the training & testing accuracies, beginning from
#50%

train_accuracies=[0.5]
test_accuracies=[0.5]


In [ ]:
#looping over some values of max_depths

for d in range(1,25):
    classifier1= tree.DecisionTreeClassifier(max_depth=d)
    classifier1.fit(df1_train[cols],df1_train.quality)
    
    train_predictions=classifier1.predict(df1_train[cols])
    test_predictions=classifier1.predict(df1_test[cols])
    
    train_acc= metrics.accuracy_score(df1_train.quality,train_predictions)
    test_acc = metrics.accuracy_score(df1_test.quality,test_predictions)
    
    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)
    
    


In [ ]:
#creating plots using matplotlib and seaborn
plt.figure(figsize=(12,5))
sns.set_style("whitegrid")
plt.plot(train_accuracies,label="Training Accuracy")
plt.plot(test_accuracies,label="Testing Accuracy")

plt.legend(loc="upper left", prop={'size':15})
plt.xticks(range(0,26,5))
plt.xlabel("max_depth",size=20)
plt.ylabel("accuracy",size=20)
plt.show()

So clearly the graph above indicates a clear case of overfitting.Here the model fits perfectly on the training data and performs poorly when it comes to the test data set.
Overfitting is observed as a case where test loss increases as we keep increasing training loss.
As the maximum depth tends to increase the model performs badly, and training loss increases.

In [ ]:
print("The train accuracies are given as :")
print(train_accuracies)

In [ ]:
print("The corresponding testing accuracies are given as :")
print(test_accuracies)


Determining correlation between various input features.

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(df1.corr(),annot=True)
plt.show()

Box plot based stastical inferences.

In [ ]:
#Plot a boxplot to check for Outliers
#Target variable is Quality. So will plot a boxplot each column against target variable
sns.boxplot('quality', 'fixed acidity', data = df1)

In [ ]:
sns.boxplot('quality', 'volatile acidity', data = df1)

In [ ]:
sns.boxplot('quality', 'citric acid', data = df1)

In [ ]:
sns.boxplot('quality', 'chlorides', data = df1)

In [ ]:
sns.boxplot('quality', 'free sulfur dioxide', data = df1)

In [ ]:
sns.boxplot('quality', 'total sulfur dioxide', data = df1)

In [ ]:
sns.boxplot('quality', 'density', data = df1)

In [ ]:
sns.boxplot('quality', 'pH', data = df1)

In [ ]:
sns.boxplot('quality', 'sulphates', data = df1)

In [ ]:
sns.boxplot('quality', 'alcohol', data = df1)

In [ ]:
#To ostify our arguement lets look at the distribution of quality labels in the given data.
q= sns.countplot(x='quality',data=df1)
q.set_xlabel("quality", fontsize=20)
q.set_ylabel("count", fontsize=20)



We can see here the distribution of data is very unbalanced,and thus the results.

We now move towards the approach of Cross-Validating the entire data set.
The script for which can be found below.


import pandas as pd
from sklearn import model_selection

if __name__=="__main__":
    df= pd.read_csv("winequality-red.csv")
    
    df["kfold"]=-1
    df=df.sample(frac=1).reset_index(drop=True)
    
    y=df.quality.values
    
    kf=model_selection.StratifiedKFold(n_splits=5)
    
    for f,(t_,v_) in enumerate(kf.split(X=df,y=y)):
        df.loc[v_,'kfold']=f
    
    df.to_csv("Train_folds_winequality-red.csv", index=False)
    

In [ ]:
df2= pd.read_csv('/kaggle/input/folds-winequality/Train_folds_winequality-red.csv')

Here I have created and uploaded a new Stratified-KFolded data set, with 5 splits.
In this approach,the performance is measured using this data now where out of 5 folds , 4 are always used as
training set and 1 being used as testing set for each iteration upon a particualr fold.

Clearly the problem can be classified both as a regression or a classification based tasks.For simplicity I have tried to gain insights by using some of the typical classification based techniques.
Following classification procedures are being employed here using the Stratified KFold Cross Validation:
1. Decision Tree classification
2. Random Forest
3. SVC Classifier
4. Multi Layer Perceptron based Neural network
5. Gradient Boosting Classifier
6. K Nearest Neighbour classifier
7. Extra Trees based classification




In [ ]:
def help_Decision_tree(fold):
    df2= pd.read_csv('/kaggle/input/folds-winequality/Train_folds_winequality-red.csv')
    qual_map={
    3:0,
    4:1,
    5:2,
    6:3,
    7:4,
    8:5
    }
    df2.loc[:,"quality"]=df2.quality.map(qual_map)
    features=[
        f for f in df2.columns if f not in("quality","kfold")
        ]
    for col in features:
        df2.loc[:,col]=df2[col].astype(str).fillna("NONE")
    
    df2_train=df2[df2.kfold!=fold].reset_index(drop=True)
    df2_valid=df2[df2.kfold==fold].reset_index(drop=True)
    
    
    x_train= df2_train.drop("quality",axis=1).values
    y_train= df2_train.quality.values
    
    x_valid= df2_valid.drop("quality",axis=1).values
    y_valid= df2_valid.quality.values
    
    model= tree.DecisionTreeClassifier()
    model.fit(x_train,y_train)
    
    valid_preds=model.predict(x_valid)
    accur= metrics.accuracy_score(y_valid,valid_preds)
    print(f"Fold = {fold}, Accuracy = {accur}")
    

In [ ]:
for i in range(0,5):
    print(help_Decision_tree(i))
    print()

Clearly an average of around 60% veracity.

In [ ]:
def help_Random_Forest(fold):
    df2= pd.read_csv('/kaggle/input/folds-winequality/Train_folds_winequality-red.csv')
    qual_map={
    3:0,
    4:1,
    5:2,
    6:3,
    7:4,
    8:5
    }
    df2.loc[:,"quality"]=df2.quality.map(qual_map)
    features=[
        f for f in df2.columns if f not in("quality","kfold")
        ]
    for col in features:
        df2.loc[:,col]=df2[col].astype(str).fillna("NONE")
    
    df2_train=df2[df2.kfold!=fold].reset_index(drop=True)
    df2_valid=df2[df2.kfold==fold].reset_index(drop=True)
    
    
    x_train= df2_train.drop("quality",axis=1).values
    y_train= df2_train.quality.values
    
    x_valid= df2_valid.drop("quality",axis=1).values
    y_valid= df2_valid.quality.values
    
    model2= RandomForestClassifier()
    model2.fit(x_train,y_train)
    
    valid_preds=model2.predict(x_valid)
    accur= metrics.accuracy_score(y_valid,valid_preds)
    print(f"Fold = {fold}, Accuracy = {accur}")
    

In [ ]:
for i in range(0,5):
    print(help_Random_Forest(i))
    print()

Random forest outperforms every other algorithm as we see, clearly dealing well with the unbalanced data and the overfitting issue.Clearly an average of around 70% veracity.

In [ ]:
def help_svc(fold):
    df2= pd.read_csv('/kaggle/input/folds-winequality/Train_folds_winequality-red.csv')
    qual_map={
    3:0,
    4:1,
    5:2,
    6:3,
    7:4,
    8:5
    }
    df2.loc[:,"quality"]=df2.quality.map(qual_map)
    features=[
        f for f in df2.columns if f not in("quality","kfold")
        ]
    for col in features:
        df2.loc[:,col]=df2[col].astype(str).fillna("NONE")
    
    df2_train=df2[df2.kfold!=fold].reset_index(drop=True)
    df2_valid=df2[df2.kfold==fold].reset_index(drop=True)
    
    
    x_train= df2_train.drop("quality",axis=1).values
    y_train= df2_train.quality.values
    
    x_valid= df2_valid.drop("quality",axis=1).values
    y_valid= df2_valid.quality.values
    
    model3= SVC()
    model3.fit(x_train,y_train)
    
    valid_preds=model3.predict(x_valid)
    accur= metrics.accuracy_score(y_valid,valid_preds)
    print(f"Fold = {fold}, Accuracy = {accur}")
    

In [ ]:
for i in range(0,5):
    print(help_svc(i))
    print()

Clearly an average of around 50% veracity.

In [ ]:
def help_mlp(fold):
    df2= pd.read_csv('/kaggle/input/folds-winequality/Train_folds_winequality-red.csv')
    qual_map={
    3:0,
    4:1,
    5:2,
    6:3,
    7:4,
    8:5
    }
    df2.loc[:,"quality"]=df2.quality.map(qual_map)
    features=[
        f for f in df2.columns if f not in("quality","kfold")
        ]
    for col in features:
        df2.loc[:,col]=df2[col].astype(str).fillna("NONE")
    
    df2_train=df2[df2.kfold!=fold].reset_index(drop=True)
    df2_valid=df2[df2.kfold==fold].reset_index(drop=True)
    
    
    x_train= df2_train.drop("quality",axis=1).values
    y_train= df2_train.quality.values
    
    x_valid= df2_valid.drop("quality",axis=1).values
    y_valid= df2_valid.quality.values
    
    model4= MLPClassifier()
    model4.fit(x_train,y_train)
    
    valid_preds=model4.predict(x_valid)
    accur= metrics.accuracy_score(y_valid,valid_preds)
    print(f"Fold = {fold}, Accuracy = {accur}")
    

In [ ]:
for i in range(0,5):
    print(help_mlp(i))
    print()

Clearly an average of around 55% veracity.

In [ ]:
def help_gbc(fold):
    df2= pd.read_csv('/kaggle/input/folds-winequality/Train_folds_winequality-red.csv')
    qual_map={
    3:0,
    4:1,
    5:2,
    6:3,
    7:4,
    8:5
    }
    df2.loc[:,"quality"]=df2.quality.map(qual_map)
    features=[
        f for f in df2.columns if f not in("quality","kfold")
        ]
    for col in features:
        df2.loc[:,col]=df2[col].astype(str).fillna("NONE")
    
    df2_train=df2[df2.kfold!=fold].reset_index(drop=True)
    df2_valid=df2[df2.kfold==fold].reset_index(drop=True)
    
    
    x_train= df2_train.drop("quality",axis=1).values
    y_train= df2_train.quality.values
    
    x_valid= df2_valid.drop("quality",axis=1).values
    y_valid= df2_valid.quality.values
    
    model5= GradientBoostingClassifier()
    model5.fit(x_train,y_train)
    
    valid_preds=model5.predict(x_valid)
    accur= metrics.accuracy_score(y_valid,valid_preds)
    print(f"Fold = {fold}, Accuracy = {accur}")
    

In [ ]:
for i in range(0,5):
    print(help_gbc(i))
    print()

Clearly an average of around 65% veracity.

In [ ]:
def help_Knn(fold):
    df2= pd.read_csv('/kaggle/input/folds-winequality/Train_folds_winequality-red.csv')
    qual_map={
    3:0,
    4:1,
    5:2,
    6:3,
    7:4,
    8:5
    }
    df2.loc[:,"quality"]=df2.quality.map(qual_map)
    features=[
        f for f in df2.columns if f not in("quality","kfold")
        ]
    for col in features:
        df2.loc[:,col]=df2[col].astype(str).fillna("NONE")
    
    df2_train=df2[df2.kfold!=fold].reset_index(drop=True)
    df2_valid=df2[df2.kfold==fold].reset_index(drop=True)
    
    
    x_train= df2_train.drop("quality",axis=1).values
    y_train= df2_train.quality.values
    
    x_valid= df2_valid.drop("quality",axis=1).values
    y_valid= df2_valid.quality.values
    
    model6= KNeighborsClassifier()
    model6.fit(x_train,y_train)
    
    valid_preds=model6.predict(x_valid)
    accur= metrics.accuracy_score(y_valid,valid_preds)
    print(f"Fold = {fold}, Accuracy = {accur}")
    

In [ ]:
for i in range(0,5):
    print(help_Knn(i))
    print()

Clearly an average of around 50% veracity.

In [ ]:
def help_ExtraTreeClassifier(fold):
    df2= pd.read_csv('/kaggle/input/folds-winequality/Train_folds_winequality-red.csv')
    qual_map={
    3:0,
    4:1,
    5:2,
    6:3,
    7:4,
    8:5
    }
    df2.loc[:,"quality"]=df2.quality.map(qual_map)
    features=[
        f for f in df2.columns if f not in("quality","kfold")
        ]
    for col in features:
        df2.loc[:,col]=df2[col].astype(str).fillna("NONE")
    
    df2_train=df2[df2.kfold!=fold].reset_index(drop=True)
    df2_valid=df2[df2.kfold==fold].reset_index(drop=True)
    
    
    x_train= df2_train.drop("quality",axis=1).values
    y_train= df2_train.quality.values
    
    x_valid= df2_valid.drop("quality",axis=1).values
    y_valid= df2_valid.quality.values
    
    model7= ExtraTreeClassifier()
    model7.fit(x_train,y_train)
    
    valid_preds=model7.predict(x_valid)
    accur= metrics.accuracy_score(y_valid,valid_preds)
    print(f"Fold = {fold}, Accuracy = {accur}")
    

In [ ]:
for i in range(0,5):
    print(help_ExtraTreeClassifier(i))
    print()

Clearly an average of around 56% veracity.

Therefore, random forest algorithm performed the best in this multiclass classification task, where I have considered the originally provided data by using a more elaborative approach of using Stratified KFold Cross Validation.Thereby accounting well with the issue of Overfitting. 


Kindly hit the upvote button :) 
    AND support my work !